In [1]:
import time
import cv2
import mediapipe as mp
import handtrackingmodule as htm
import os
import numpy as np

In [2]:
folder="header"
mylist=os.listdir(folder)
#print(mylist)
overlay=[]
for imgpath in mylist:
    image=cv2.imread(f'{folder}/{imgpath}')
    overlay.append(image)

In [3]:
cap=cv2.VideoCapture(0)
cap.set(3,1100)
cap.set(4,420)
detector=htm.handdetector()
header=overlay[0]
drawcolor=(0,0,255)
brushthickness=15
eraserthickness=100
imgcanvas=np.zeros((720,1280,3),dtype=np.uint8)
xp,yp=0,0
while True:
    # image
    success,img=cap.read()
    img=cv2.flip(img,1)

    # find landmarks
    img=detector.findHands(img)
    lmlist=detector.findposition(img,draw=False)
    if len(lmlist)!=0:
        x1,y1=lmlist[8][1],lmlist[8][2]
        x2,y2=lmlist[12][1],lmlist[12][2]
    
        # check fingersup
        fingers=detector.fingersup()
        #print(fingers)

        # if selection mode-two fingers up
        if fingers[1]=='1'and fingers[2]=='1':
            xp,yp=0,0
            if y1<110:
                if 125<x1<165:
                    header=overlay[0]
                    drawcolor=(0,0,255)
                elif 185<x1<235:
                    header=overlay[1]
                    drawcolor=(255,255,255)
                elif  295<x1<365:
                    header=overlay[2]
                    drawcolor=(255,0,0)
                elif  480<x1<510:
                    header=overlay[3]
                    drawcolor=(0,255,0)
                elif 630<x1<700:
                    header=overlay[4]
                    drawcolor=(0,0,0)
            cv2.rectangle(img,(x1,y1-25),(x2,y2+25),drawcolor,cv2.FILLED)
        

        # drwaing mode- index finger up
        if fingers[1]=='1' and fingers[2]=='0':
            cv2.circle(img,(x1,y1),15,drawcolor,cv2.FILLED)
            #print("drawing")
            if xp==0 and yp==0:
                xp,yp=x1,y1
            if drawcolor==(0,0,0):
                cv2.line(img,(xp,yp),(x1,y1),drawcolor,eraserthickness)
                cv2.line(imgcanvas,(xp,yp),(x1,y1),drawcolor,eraserthickness)
            else:
                cv2.line(img,(xp,yp),(x1,y1),drawcolor,brushthickness)
                cv2.line(imgcanvas,(xp,yp),(x1,y1),drawcolor,brushthickness)
            xp,yp=x1,y1

    imggray=cv2.cvtColor(imgcanvas,cv2.COLOR_BGR2GRAY)
    _,imginv=cv2.threshold(imggray,50,255,cv2.THRESH_BINARY_INV)
    imginv=cv2.cvtColor(imginv,cv2.COLOR_GRAY2BGR)
    img=cv2.bitwise_and(img,imginv)
    img=cv2.bitwise_or(img,imgcanvas)


    #setting header
    h,w,c=header.shape
    img[0:h,0:w]=header
    cv2.imshow('img',img)
    #cv2.imshow('canvas',imgcanvas)
    cv2.waitKey(1)


